In [0]:
import numpy as np
import random
import torch
random.seed(1234)
torch.manual_seed(1234)

In [0]:
!wget -q -nc https://raw.githubusercontent.com/jhu-intro-hlt/jhu-intro-hlt.github.io/master/data-phone-recognitiona/dev_feats.mat.npy
!wget -q -nc https://raw.githubusercontent.com/jhu-intro-hlt/jhu-intro-hlt.github.io/master/data-phone-recognitiona/train_feats.mat.npy
!wget -q -nc https://raw.githubusercontent.com/jhu-intro-hlt/jhu-intro-hlt.github.io/master/data-phone-recognitiona/test_feats.mat.npy
!wget -q -nc https://raw.githubusercontent.com/jhu-intro-hlt/jhu-intro-hlt.github.io/master/data-phone-recognitiona/dev_labels.mat.npy
!wget -q -nc https://raw.githubusercontent.com/jhu-intro-hlt/jhu-intro-hlt.github.io/master/data-phone-recognitiona/train_labels.mat.npy
!wget -q -nc https://raw.githubusercontent.com/jhu-intro-hlt/jhu-intro-hlt.github.io/master/data-phone-recognitiona/test_labels.mat.npy

In [0]:
def get_labels(data):
    label_dict = {}
    for y in data:
        label_dict[y] = label_dict.get(y, len(label_dict))
    return label_dict


def load_npy():
    train_feats = np.load('train_feats.mat.npy', allow_pickle=True)
    train_labels = np.load('train_labels.mat.npy', allow_pickle=True)
    label_idx = get_labels(train_labels)
    test_feats = np.load('test_feats.mat.npy', allow_pickle=True)
    test_labels = np.load('test_labels.mat.npy', allow_pickle=True)
    dev_feats = np.load('dev_feats.mat.npy', allow_pickle=True)
    dev_labels = np.load('dev_labels.mat.npy', allow_pickle=True)
    return label_idx, (train_feats, train_labels), (dev_feats, dev_labels), (test_feats, test_labels)


In [0]:
label_dict, train, dev, test = load_npy()

#Display the shape of the features and labels
print(train[0].shape, len(train[1]))
print(dev[0].shape, len(dev[1]))
print(test[0].shape, len(test[1]))

#Display the first 40 labels
print(train[1][:300])
#Dispplay the first 40 speech features
print(train[0][:40])

(200000, 39) 200000
(10000, 39) 10000
(10000, 39) 10000
['sil' 'sil' 'sil' 'sil' 'sil' 'sil' 'sil' 'sil' 'sil' 'sil' 'sil' 'sil'
 'sil' 'ax' 'ax' 'ax' 'ax' 'ax' 'ax' 's' 's' 's' 's' 's' 's' 's' 's' 's'
 's' 's' 's' 's' 's' 's' 's' 's' 'uw' 'uw' 'uw' 'uw' 'uw' 'uw' 'uw' 'uw'
 'uw' 'm' 'm' 'm' 'm' 'm' 'm' 'm' 'm' 'm' 'f' 'f' 'f' 'f' 'f' 'f' 'f' 'f'
 'f' 'f' 'f' 'f' 'f' 'ao' 'ao' 'ao' 'ao' 'ao' 'ao' 'r' 'r' 'r' 'r' 'r'
 'ix' 'ix' 'ix' 'vcl' 'vcl' 'vcl' 'vcl' 'vcl' 'z' 'z' 'z' 'z' 'z' 'z' 'z'
 'z' 'ae' 'ae' 'ae' 'ae' 'ae' 'ae' 'ae' 'ae' 'ae' 'ae' 'm' 'm' 'm' 'm'
 'cl' 'cl' 'cl' 'p' 'p' 'p' 'p' 'uh' 'uh' 'uh' 'l' 'l' 'l' 'l' 'l' 'l' 'l'
 'l' 'l' 'l' 'l' 'l' 'l' 'ax' 'ax' 'ax' 'ax' 'ax' 'ax' 'ax' 's' 's' 's'
 's' 's' 's' 's' 's' 's' 's' 's' 's' 's' 'ix' 'ix' 'ix' 'ix' 'ix' 'cl'
 'cl' 'cl' 'ch' 'ch' 'ch' 'ch' 'ch' 'ch' 'ch' 'ch' 'ch' 'ch' 'ch' 'uw'
 'uw' 'uw' 'uw' 'uw' 'uw' 'uw' 'uw' 'uw' 'ey' 'ey' 'ey' 'ey' 'ey' 'ey'
 'ey' 'ey' 'ey' 'ey' 'sh' 'sh' 'sh' 'sh' 'sh' 'sh' 'sh' 'sh' 'sh' 'sh'
 'sh

In [0]:
def batchify(data_feats, data_labels, batch_size, label_dict, window=5, to_cuda=False):
    batched_tdata = []
    curr_batch = []
    fz = np.zeros((window, 39))
    bz = np.zeros((window, 39))
    fl = ['sil'] * window
    bl = ['sil'] * window
    data_labels = fl + data_labels.tolist() + bl
    data_feats = np.concatenate((fz, data_feats, bz))
    for i in range(window, len(data_labels) - window):
        x = data_feats[i - window: i + window + 1]
        y = data_labels[i]
        tx = torch.Tensor(x).unsqueeze(0) # shape should be (1, 39, 2window)
        ty = torch.Tensor([label_dict[y]]) # shape should be (1, 1)
        if len(curr_batch) < batch_size:
            #if y != 'sil':
            curr_batch.append((tx, ty))
        else:
            _tx, _ty = zip(*curr_batch)
            b_tx = torch.cat(_tx, dim=0)
            b_ty = torch.cat(_ty, dim=0)
            if to_cuda:
                b_tx, b_ty = b_tx.cuda(), b_ty.cuda()
            batched_tdata.append((b_ty, b_tx))
            curr_batch = []
    if len(curr_batch) > 0:
        _tx, _ty = zip(*curr_batch)
        b_tx = torch.cat(_tx, dim=0)
        b_ty = torch.cat(_ty, dim=0)
        if to_cuda:
            b_tx, b_ty = b_tx.cuda(), b_ty.cuda()
        batched_tdata.append((b_ty, b_tx))
    return batched_tdata

In [0]:
window=0
batched_train_0 = batchify(train[0], train[1], 2000, label_dict, window, True)
batched_dev_0 = batchify(dev[0], dev[1], 2000, label_dict, window, True)
batched_test_0 = batchify(test[0], test[1], 2000, label_dict, window, True)

## Model

In [0]:
class MLP_Simple(torch.nn.Module):
    def __init__(self,
                 hidden_size,
                 num_labels):
        super().__init__()

        self.layer0 = torch.nn.Linear(1 * 39, hidden_size)

        self.layer1 = torch.nn.ReLU(inplace=False)
        self.layer2 = torch.nn.Linear(hidden_size, hidden_size)
        self.layer3 = torch.nn.ReLU(inplace=False)
        self.layer4 = torch.nn.Linear(hidden_size, hidden_size)

        self.final_layer = torch.nn.Linear(hidden_size, num_labels)

    def forward(self, x):
        """Generate output distribution and argmax
        Args:
            x: num frames of 39-dimensional feature vector (MFCC features)
        Return:
            dist: log probability for each output class
            pred: the label with highest log probability
        """
        batch_size, frames, features = x.shape
        x = x.squeeze(1)
        x = self.layer0(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        y_hat = self.final_layer(x) 
        _, y_pred = y_hat.max(dim=-1) 
        return y_hat, y_pred

In [0]:
model_simple = MLP_Simple(hidden_size=300, num_labels=len(label_dict))
print(model_simple)
print('num parameters:', sum([p.numel() for p in model_simple.parameters()]))

MLP_Simple(
  (layer0): Linear(in_features=39, out_features=300, bias=True)
  (layer1): ReLU()
  (layer2): Linear(in_features=300, out_features=300, bias=True)
  (layer3): ReLU()
  (layer4): Linear(in_features=300, out_features=300, bias=True)
  (final_layer): Linear(in_features=300, out_features=48, bias=True)
)
num parameters: 207048


## Training

In [0]:
def train_model(model, batched_train, batched_dev, batched_test, max_epoch=20):
    model = model.cuda()
    loss = torch.nn.CrossEntropyLoss(reduction='mean')
    optim = torch.optim.Adam(model.parameters())
  
    for epoch in range(max_epoch):
        random.shuffle(batched_train)
        train_loss = []
        train_acc = []
        model.train()
        for batch in batched_train:
            optim.zero_grad()
            y, x = batch
            y_hat, y_pred = model(x)
            batch_loss = loss(y_hat, y.long())
            batch_loss.backward()
            optim.step()
            batch_acc = (y_pred == y.long()).sum().item() / y.numel()
            train_loss.append(batch_loss.item())
            train_acc.append(batch_acc)
        _loss = sum(train_loss) / len(train_loss)
        _acc = sum(train_acc) / len(train_acc)
        print(f"Epoch {epoch}")
        print(f"train loss {_loss:.4f} train_acc {_acc:.4f}")
        dev_acc = []
        model.eval()
        for batch in batched_dev:
            y, x = batch
            with torch.no_grad():
                y_hat, y_pred = model(x)
                batch_acc = (y_pred == y.long()).sum().item() / y.numel()
                dev_acc.append(batch_acc)
        _acc = sum(dev_acc) / len(dev_acc)
        print(f"dev_acc {_acc:.4f}")
    test_acc = []
    model.eval()
    for batch in batched_test:
        y, x = batch
        with torch.no_grad():
            y_hat, y_pred = model(x)
            batch_acc = (y_pred == y.long()).sum().item() / y.numel()
            test_acc.append(batch_acc)
    _acc = sum(test_acc) / len(test_acc)
    print(f"training completed.\n")
    print(f"test_acc {_acc:.4f}")

train_model(model_simple, batched_train_0, batched_dev_0, batched_test_0)

Epoch 0
train loss 1.9897 train_acc 0.4238
dev_acc 0.4687
Epoch 1
train loss 1.5915 train_acc 0.5111
dev_acc 0.5212
Epoch 2
train loss 1.4451 train_acc 0.5498
dev_acc 0.5387
Epoch 3
train loss 1.3641 train_acc 0.5699
dev_acc 0.5397
Epoch 4
train loss 1.3005 train_acc 0.5871
dev_acc 0.5488
Epoch 5
train loss 1.2546 train_acc 0.5979
dev_acc 0.5606
Epoch 6
train loss 1.2178 train_acc 0.6073
dev_acc 0.5691
Epoch 7
train loss 1.1865 train_acc 0.6171
dev_acc 0.5738
Epoch 8
train loss 1.1580 train_acc 0.6248
dev_acc 0.5743
Epoch 9
train loss 1.1367 train_acc 0.6295
dev_acc 0.5880
Epoch 10
train loss 1.1037 train_acc 0.6400
dev_acc 0.5808
Epoch 11
train loss 1.0829 train_acc 0.6455
dev_acc 0.5805
Epoch 12
train loss 1.0747 train_acc 0.6468
dev_acc 0.5820
Epoch 13
train loss 1.0568 train_acc 0.6529
dev_acc 0.5957
Epoch 14
train loss 1.0374 train_acc 0.6588
dev_acc 0.5958
Epoch 15
train loss 1.0225 train_acc 0.6622
dev_acc 0.5813
Epoch 16
train loss 1.0072 train_acc 0.6676
dev_acc 0.5953
Epoch 1

# Adding Context

In [0]:
window=5
batched_train_5 = batchify(train[0], train[1], 2000, label_dict, window, True)
batched_dev_5 = batchify(dev[0], dev[1], 2000, label_dict, window, True)
batched_test_5 = batchify(test[0], test[1], 2000, label_dict, window, True)

In [0]:
class MLP_Context(torch.nn.Module):
    def __init__(self,
                 hidden_size,
                 num_labels):
        super().__init__()
        self.layer0 = torch.nn.Linear(11 * 39, hidden_size)

        self.layer1 = torch.nn.Linear(hidden_size, hidden_size)
        self.layer2 = torch.nn.ReLU(inplace=False)

        self.final_layer = torch.nn.Linear(hidden_size, num_labels)

    def forward(self, x):
        """Generate output distribution and argmax
        Args:
            x: num frames of 39-dimensional feature vector (MFCC features)
        Return:
            dist: log probability for each output class
            pred: the label with highest log probability
        """
        batch_size, frames, features = x.shape
        x = x.view(batch_size, -1)
        x = self.layer0(x)

        x = self.layer1(x)
        x = self.layer2(x)

        y_hat = self.final_layer(x)
        _, y_pred = y_hat.max(dim=-1)
        return y_hat, y_pred

In [0]:
model_context = MLP_Context(hidden_size=300, num_labels=len(label_dict))
train_model(model_context, batched_train_5, batched_dev_5, batched_test_5)

Epoch 0
train loss 1.7077 train_acc 0.5045
dev_acc 0.5544
Epoch 1
train loss 1.3087 train_acc 0.5927
dev_acc 0.5938
Epoch 2
train loss 1.1920 train_acc 0.6260
dev_acc 0.6018
Epoch 3
train loss 1.1221 train_acc 0.6440
dev_acc 0.5986
Epoch 4
train loss 1.0798 train_acc 0.6553
dev_acc 0.6054
Epoch 5
train loss 1.0509 train_acc 0.6634
dev_acc 0.6077
Epoch 6
train loss 1.0229 train_acc 0.6730
dev_acc 0.6185
Epoch 7
train loss 0.9916 train_acc 0.6806
dev_acc 0.6230
Epoch 8
train loss 0.9730 train_acc 0.6861
dev_acc 0.6144
Epoch 9
train loss 0.9606 train_acc 0.6899
dev_acc 0.6238
Epoch 10
train loss 0.9460 train_acc 0.6946
dev_acc 0.6193
Epoch 11
train loss 0.9338 train_acc 0.6974
dev_acc 0.6397
Epoch 12
train loss 0.9243 train_acc 0.7011
dev_acc 0.6183
Epoch 13
train loss 0.9107 train_acc 0.7029
dev_acc 0.6092
Epoch 14
train loss 0.8853 train_acc 0.7112
dev_acc 0.6262
Epoch 15
train loss 0.8871 train_acc 0.7087
dev_acc 0.6234
Epoch 16
train loss 0.8839 train_acc 0.7121
dev_acc 0.6188
Epoch 1